In [1]:
from selenium import webdriver
from model import supportVectorMachineModel
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

common_terms=["directory","faculty","faculty-staff","faculty-directory","people","people-page","staff",
              "teaching-faculty","about-people","members","all-faculty-staff","faculty-and-staff",
              "faculty-list","professors","faculty-profiles"]

class scraper:
    def __init__(instance):
        options=Options()
        options.headless=True
        instance.driver=webdriver.Chrome("./chromedriver.exe",options=options)
        instance.classification=supportVectorMachineModel()
        instance.classification.start()

    def check_link(instance,url):
        for part in [part for part in url.split("/") if part]:
            for term in common_terms:
                if term in part:
                    if len(part)!=len(term):
                        return False

        return True

    def get_all_directories(instance,base_link):
        results=[]
        
        instance.driver.get(base_link)
        res_html=instance.driver.execute_script("return document.body.innerHTML")
        soup=BeautifulSoup(res_html,"html.parser")

        for element in soup.find_all("a"):
            result=base_link+element.get("href","")
            if instance.check_link(result) and instance.classification.analyze(result)=="1" and (result not in results):
                results.append(result)

        return results

    def get_all_faculties(instance,base_link,test_link):
        results=[]
        
        instance.driver.get(test_link)
        res_html=instance.driver.execute_script("return document.body.innerHTML")
        soup=BeautifulSoup(res_html,"html.parser")

        for element in soup.find_all("a"):
            result=base_link+element.get("href","")
            if (result!=test_link) and instance.classification.analyze(result)=="1" and (result not in results):
                results.append(result)

        return results

    def start_scraping(instance,base_link):
        results=[]
        total_size=100
        
        for element in instance.get_all_directories(base_link):
            if (len(results)<total_size):
                for current in instance.get_all_faculties(base_link,element):
                    if current not in results:
                        results.append(current)

        return results

In [2]:
scraper1=scraper()

url="https://cse.osu.edu/"
directory_urls=scraper1.get_all_directories(url)
faculty_urls=scraper1.start_scraping(url)

# write directory urls to directory_urls.txt
file=open("directory_urls.txt","w")
for url in directory_urls:
    file.write(url)
    file.write("\n")
file.close()

# write faculty urls to faculty_urls.txt
file=open("faculty_urls.txt","w")
for url in faculty_urls:
    file.write(url)
    file.write("\n")
file.close()

<ipython-input-1-c537e5247d18>:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  instance.driver=webdriver.Chrome("./chromedriver.exe",options=options)


In [3]:
print('Successfully found '+str(len(directory_urls))+' directory urls and '+str(len(faculty_urls))+' faculty urls')

Successfully found 17 directory urls and 102 faculty urls
